# **Create Single Monkey Surfaces**  

This notebook will guide you through the procedure to create single monkey surface representations in four steps (some are optional):  

1. **(OPTIONAL)** Average multiple T1 scans from the same animal. (takes seconds)
2. Segment the (averaged) T1 of an individual animal using the NMT template as a prior. (takes many hours)
3. Create surfaces and flatmaps using Freesurfer. (takes up to to several hours depending on segmentation quality at the start)
4. **(OPTIONAL)** Create additional surfaces with Connectome Workbench. (takes minutes)

## Initiate 
Setting a few environment variables here so we can actually run code from this notebook

In [10]:
SUBJ=NMT  # Name of the animal you are going to process
NMT_path=/NHP_MRI/Template/NMT # path to the template folder
startpath=pwd # current directory

MRISreadVertexPosition(white): could not open file /NHP_MRI/Template/NMT/single_subject_scans/NMT/fsSurf/temp/lh.white
No such file or directory
could not load surface from white
No such file or directory
te): could not open file /NHP_MRI/Template/NMT/single_subject_scans/NMT/fsSurf/temp/lh.white
MRISreadVertexPosition(white): could not open file /NHP_MRI/Template/NMT/single_subject_scans/NMT/fsSurf/temp/rh.white
No such file or directory
could not load surface from white
No such file or directory
te): could not open file /NHP_MRI/Template/NMT/single_subject_scans/NMT/fsSurf/temp/rh.white
Application asked to unregister timer 0x43000003 which is not registered in this thread. Fix application.
[1]+  Done                    freeview -v ${fsSurf_mgz}/brain.mgz -v ${fsSurf_mgz}/wm.mgz -f ${fsSurf_temp}/lh.smoothwm ${fsSurf_temp}/lh.inflated ${fsSurf_temp}/rh.smoothwm ${fsSurf_temp}/rh.inflated


----
----
## **Step 1 (OPTIONAL): Average multiple T1 scans**

----
----

### Requirements:
- Freesurfer installation (https://surfer.nmr.mgh.harvard.edu/)
- NIH Macaque Template (https://github.com/jms290/NMT)
- One or more high resolution anatomical scans

- The `average_multiple_t1.sh ` script  
  
### Procedure
- Create a folder for your subject in the NIH Macaque Template folder as 

`<>/NHP_MRI/Template/NMT/single_subject_scans/<SUBJECT>`  

- Inside this folder, create a subfolder with all individual T1 scan for this subject, e.g. 

`<>/NHP_MRI/Template/NMT/single_subject_scans/<SUBJECT>/<T1_FLD>/*.nii.gz`  

- Copy the script `average_multiple_t1.sh` to the `<SUBJECT>` folder as well.  

- Run this script with the name of the `<T1_FLD>` as input:  

`$ ./average_multiple_t1.sh <T1_FLD>`  


- The script will:  

    - Correct for having the monkey in the sphinx position  
    - Resample each T1 to 0.5 mm isotropic voxels  
    - Reorient the colume for correct display in FSL (e.g., with FSLEyes)  
    - Normalize the contrast gradient  
    - Average the multiple T1's together using motion correction to account for small differences  
  
  
- When the script is done, there should be a `<SUBJECT>.nii.gz` file in the root `<SUBJECT>` folder.

In [ ]:
# Get path and make folders
NMT_ss_path=${NMT_path}/single_subject_scans/${SUBJ}
mkdir -p ${NMT_ss_path}
mkdir -p ${NMT_path}/single_subject_scans/${SUBJ}/T1s
echo Copy the T1 files to ${NMT_path}/single_subject_scans/${SUBJ}/T1s

In [ ]:
# Copy averaging script
cp ${NMT_path}/single_subject_scans/average_multiple_t1.sh \
    ${NMT_path}/single_subject_scans/${SUBJ}/
cd ${NMT_path}/single_subject_scans/${SUBJ}/
# Run script
./average_multiple_t1.sh T1s

----
----
## **Step 2: Segment the single subject anatomy using the NMT template as a prior**

----
----

### Requirements
  - NIH Macaque Template (https://github.com/jms290/NMT)  
  - ANTS (http://stnava.github.io/ANTs/)
  - AFNI (https://afni.nimh.nih.gov/)
 
### Procedure
This will segment and process the single-subject anatomical and warp the D99 atlas (https://afni.nimh.nih.gov/Macaque) to your individual anatomy.  

- Navigate to `<>/NHP_MRI/Template/NMT/single_subject_scans` in your terminal
  
- There should be a folder `<SUBJECT>` containing a T1 named `<SUBJECT>.nii.gz` (Step 1 produces this)
  
- Run the `align_and_process_singlesubject.sh` script with input `<SUBJECT>`:
      
- The following 2 steps can be executed together by running: `$ ./create_ss_atlas.sh <SUBJECT>`  

    - Extracting single ROI masks from the warped atlas: `./extract_ROI.sh <SUBJECT>`  

    - Creating a label-file that makes the warped atlas usable in FSLEyes: `./create_xml.sh <SUBJECT>`


In [ ]:
# Go to the single subject folder in NMT
cd ${NMT_path}/single_subject_scans
# Run script to align and segment. This will take a long time (~8-10 hours)
./align_and_process_singlesubject.sh ${SUBJ}

You may already want to perform some manual adjustments here (e.g., white matter tends to be overestimated in the occipital lobe). There are many ways to do this but one easy way is with `ITK-SNAP` (http://www.itksnap.org/pmwiki/pmwiki.php). Whenever you decide to make adjustments, dot it on a copy of the original results and leave the original results intact.  

In `ITK-SNAP` you can open the brain volume as the `Main Image` and load the segmentation volume as segmentation.

![ITKSNAP](pics/itksnap_loaded.png)

Voxels in the segmentation are labeled red (value 1) for CSF, green (value 2) for grey matter, and blue (value 3) for white matter. You can use the paintbrush tool to change labels and save the adjusted segmentation file.

To generate separate CSF/GM/WM masks you can run
```
fslmaths <SUBJECT>_segmentation.nii.gz -thr 0.9 -uthr 1.1 -bin <SUBJECT>_CSF.nii.gz
fslmaths <SUBJECT>_segmentation.nii.gz -thr 1.9 -uthr 2.1 -bin <SUBJECT>_GM.nii.gz
fslmaths <SUBJECT>_segmentation.nii.gz -thr 2.9 -uthr 3.1 -bin <SUBJECT>_WM.nii.gz
fslmaths <SUBJECT>_segmentation.nii.gz -thr 0.5 -uthr 3.1 -bin <SUBJECT>_brainmask.nii.gz
```

In addtion you can create atlas files so you use the warped D99 atlas for this individual in `FSLEyes` 

In [ ]:
# Create usability files for the warped D99 atlas in individual space
./create_ss_atlas ${SUBJ}

----
----
## **Step 3: Create surfaces and flatmaps using Freesurfer**

----
----

### Requirements  
- Freesurfer installation (https://surfer.nmr.mgh.harvard.edu/)
- Segmented individual subject T1

### Procedure
#### Create folders & copy files

In [11]:
# Path to the single_subjects folder in NMT where this subject's segmentation is located
#NMT_src=${NMT_path}/single_subject_scans/${SUBJ}/NMT_${SUBJ}_process 
# adjust this path if you have manually adjusted and saved to a different folder, e.g.
NMT_src=${NMT_path}/single_subject_scans/${SUBJ}

# create a folder for the freesurfer surfaces we will eventually end up with
fsSurf=${NMT_path}/single_subject_scans/${SUBJ}/fsSurf
mkdir -p ${fsSurf}

# make subfolders for structure
fsSurf_src=${fsSurf}/src
fsSurf_mgz=${fsSurf}/mgz
fsSurf_temp=${fsSurf}/temp
mkdir -p ${fsSurf_src}
mkdir -p ${fsSurf_mgz}
mkdir -p ${fsSurf_temp}

# initialize an array to be able to loop over hemispheres
HEMI=(lh rh) 

In [12]:
# copy the source files from the NMT segmentation to the freesurfer folders you just created
cp ${NMT_src}/${SUBJ}_N4.nii.gz ${fsSurf_src}/T1.nii.gz
cp ${NMT_src}/${SUBJ}_brain.nii.gz ${fsSurf_src}/brain.nii.gz
cp ${NMT_src}/${SUBJ}_brain.nii.gz ${fsSurf_src}/brainmask.nii.gz # freesurfer brainmask aren't actually masks
cp ${NMT_src}/${SUBJ}_WM.nii.gz ${fsSurf_src}/wm.nii.gz

# copy source files again but keep these in original state 
# (for others the header will be changed to 'fake' 1 mm voxels, which is necessary for freesurfer)
mkdir -p ${fsSurf_src}/org
cp ${fsSurf_src}/*.nii.gz ${fsSurf_src}/org/

#### Change headers to mimic 1mm isotropic voxels & make mgz files

In [15]:
# change headers
3drefit -xdel 1.0 -ydel 1.0 -zdel 1.0 -keepcen ${fsSurf_src}/T1.nii.gz &
3drefit -xdel 1.0 -ydel 1.0 -zdel 1.0 -keepcen ${fsSurf_src}/brain.nii.gz &
3drefit -xdel 1.0 -ydel 1.0 -zdel 1.0 -keepcen ${fsSurf_src}/brainmask.nii.gz &
3drefit -xdel 1.0 -ydel 1.0 -zdel 1.0 -keepcen ${fsSurf_src}/wm.nii.gz &
wait
# convert to mgz files for freesurfer
mri_convert -c ${fsSurf_src}/T1.nii.gz ${fsSurf_mgz}/T1.mgz &
mri_convert -c ${fsSurf_src}/brain.nii.gz ${fsSurf_mgz}/brain.mgz &
mri_convert -c ${fsSurf_src}/brain.nii.gz ${fsSurf_mgz}/brainmask.mgz &
mri_convert -c ${fsSurf_src}/brainmask.nii.gz ${fsSurf_mgz}/brainmask_binary.mgz &
mri_convert -c ${fsSurf_src}/wm.nii.gz ${fsSurf_mgz}/wm.mgz &
wait
# create brain.finalsurfs.mgz
mri_mask -T 5 ${fsSurf_mgz}/brain.mgz ${fsSurf_mgz}/brainmask.mgz ${fsSurf_mgz}/brain.finalsurfs.mgz &

ERROR: Read 176, expected 253
niiRead(): error reading from /NHP_MRI/Template/NMT/single_subject_scans/NMT/fsSurf/src/T1.nii.gz
++ 3drefit processed 1 datasets
++ 3drefit processed 1 datasets
++ 3drefit processed 1 datasets
DoAbs = 0
Found 5214430 voxels in mask (pct= 12.48)
Writing masked volume to /NHP_MRI/Template/NMT/single_subject_scans/NMT/fsSurf/mgz/brain.finalsurfs.mgz...done.
++ 3drefit processed 1 datasets
[1]   Done                    3drefit -xdel 1.0 -ydel 1.0 -zdel 1.0 -keepcen ${fsSurf_src}/T1.nii.gz
[2]   Done                    3drefit -xdel 1.0 -ydel 1.0 -zdel 1.0 -keepcen ${fsSurf_src}/brain.nii.gz
[3]   Done                    3drefit -xdel 1.0 -ydel 1.0 -zdel 1.0 -keepcen ${fsSurf_src}/brainmask.nii.gz
[4]   Done                    3drefit -xdel 1.0 -ydel 1.0 -zdel 1.0 -keepcen ${fsSurf_src}/wm.nii.gz
[5]-  Exit 1                  mri_convert -c ${fsSurf_src}/T1.nii.gz ${fsSurf_mgz}/T1.mgz
[9]+  Done                    mri_mask -T 5 ${fsSurf_mgz}/brain.mgz ${fsSurf

: 1

#### Get corpus callosum and pons voxel coordinates
These will later define the cutting planes for the surface generation so make sure that 1) the corpus callosum is in the middle, and 2) the pons coordinates are low enough to not cut through cortex. 

Corpus callosum:  
![CC](pics/CC_coordinates.png)

Pons: 
![PONS](pics/PONS_coordinates.png)

In [16]:
# Inspect volume to get voxel coordinates
freeview -v ${fsSurf_mgz}/brain.mgz &

DoAbs = 0
Found 5214430 voxels in mask (pct= 12.48)
Writing masked volume to /NHP_MRI/Template/NMT/single_subject_scans/NMT/fsSurf/mgz/brain.finalsurfs.mgz...done.
[1]+  Done                    mri_mask -T 5 ${fsSurf_mgz}/brain.mgz ${fsSurf_mgz}/brainmask.mgz ${fsSurf_mgz}/brain.finalsurfs.mgz
[1] 26548


In [27]:
#NMT
CC=(174 152 173) # corpus callosum
PONS=(174 226 139) # pons

MRISreadVertexPosition(white): could not open file /NHP_MRI/Template/NMT/single_subject_scans/NMT/fsSurf/temp/lh.white
No such file or directory
could not load surface from white
No such file or directory
te): could not open file /NHP_MRI/Template/NMT/single_subject_scans/NMT/fsSurf/temp/lh.white
MRISreadVertexPosition(white): could not open file /NHP_MRI/Template/NMT/single_subject_scans/NMT/fsSurf/temp/rh.white
No such file or directory
could not load surface from white
No such file or directory
te): could not open file /NHP_MRI/Template/NMT/single_subject_scans/NMT/fsSurf/temp/rh.white
/NHP_MRI/Template/NMT/single_subject_scans/NMT/fsSurf/mgz/wm.mgz saved successfully.
[1]+  Done                    freeview -v ${fsSurf_mgz}/brain.mgz -v ${fsSurf_mgz}/wm.mgz -f ${fsSurf_temp}/lh.smoothwm ${fsSurf_temp}/lh.inflated ${fsSurf_temp}/rh.smoothwm ${fsSurf_temp}/rh.inflated


#### Fill the white matter volume & copy for backup before fixing

In [18]:
# Fill WM
mri_fill -CV ${CC[0]} ${CC[1]} ${CC[2]} \
    -PV ${PONS[0]} ${PONS[1]} ${PONS[2]} \
    ${fsSurf_mgz}/wm.mgz ${fsSurf_mgz}/filled.mgz
# copy the original white matter before applying fixes    
cp ${fsSurf_mgz}/wm.mgz ${fsSurf_mgz}/wm_nofix.mgz

using voxel position (174, 152, 173) as corpus callosum seed point
using voxel position (174, 215, 139) as pons seed point
reading input volume...done.
searching for cutting planes...INFO: volume does not have linear_transform set nor lta is given by option.-xform.
INFO: use identity matrix as the talairach transform.
INFO: use src volume parameters for the talairach volume.
voxel to talairach voxel transform
 1.00000   0.00000   0.00000   0.00000;
 0.00000   1.00000   0.00000   0.00000;
 0.00000   0.00000   1.00000   0.00000;
 0.00000   0.00000   0.00000   1.00000;
Calculated talairach cc position (-0.25, -7.25, 25.50)
Using the seed to calculate the cutting plane
Looking for area (min, max) = (350, 1400)
area[0] = 1677 (min = 350, max = 1400), aspect = 0.42 (min = 0.10, max = 0.75)
need search nearby
using seed (174, 152, 173), TAL = (-0.2, -7.2, 25.5)
talairach voxel to voxel transform
 1.00000   0.00000   0.00000   0.00000;
 0.00000   1.00000   0.00000   0.00000;
 0.00000   0.00000

#### Tesselate volumes & fix topology (First run)

In [19]:
# Tesselate
mri_pretess ${fsSurf_mgz}/filled.mgz 255 ${fsSurf_mgz}/brain.mgz ${fsSurf_mgz}/wm_filled-pretess255.mgz &
mri_pretess ${fsSurf_mgz}/filled.mgz 127 ${fsSurf_mgz}/brain.mgz ${fsSurf_mgz}/wm_filled-pretess127.mgz &
wait
mri_tessellate ${fsSurf_mgz}/wm_filled-pretess255.mgz 255 ${fsSurf_temp}/lh.orig.nofix &
mri_tessellate ${fsSurf_mgz}/wm_filled-pretess127.mgz 127 ${fsSurf_temp}/rh.orig.nofix &
wait

# for both hemispheres
for xh in ${HEMI[@]}; do
    # create a version we can edit
    cp ${fsSurf_temp}/${xh}.orig.nofix ${fsSurf_temp}/${xh}.orig

    # post-process tesselation
    mris_extract_main_component ${fsSurf_temp}/${xh}.orig.nofix ${fsSurf_temp}/${xh}.orig.nofix
    mris_smooth -nw ${fsSurf_temp}/${xh}.orig.nofix ${fsSurf_temp}/${xh}.smoothwm.nofix
    mris_inflate ${fsSurf_temp}/${xh}.smoothwm.nofix ${fsSurf_temp}/${xh}.inflated.nofix
    mris_sphere -q ${fsSurf_temp}/${xh}.inflated.nofix ${fsSurf_temp}/${xh}.qsphere.nofix
    cp ${fsSurf_temp}/${xh}.inflated.nofix ${fsSurf_temp}/${xh}.inflated

    # fix topology
    mris_euler_number ${fsSurf_temp}/${xh}.orig
    mris_remove_intersection ${fsSurf_temp}/${xh}.orig ${fsSurf_temp}/${xh}.orig
    mris_smooth -nw ${fsSurf_temp}/${xh}.orig ${fsSurf_temp}/${xh}.smoothwm
    mris_inflate ${fsSurf_temp}/${xh}.smoothwm ${fsSurf_temp}/${xh}.inflated
done

[2] 26871
[3] 26872
Ambiguous edge configurations... 

Iteration Number : 1Ambiguous edge configurations... 

Iteration Number : 1
pass   1 (xy+):   0 found -   0 modified     |    TOTAL:   0
pass   1 (xy+):   1 found -   1 modified     |    TOTAL:   1
pass   1 (xy-):   1 found -   1 modified     |    TOTAL:   1
pass   2 (xy+):   0 found -   1 modified     |    TOTAL:   1
pass   2 (xy-):   0 found -   1 modified     |    TOTAL:   1
pass   1 (xy-):   0 found -   0 modified     |    TOTAL:   1
pass   1 (yz+):   2 found -   2 modified     |    TOTAL:   3
pass   1 (yz+):   1 found -   1 modified     |    TOTAL:   2
pass   2 (yz+):   0 found -   2 modified     |    TOTAL:   3
pass   2 (yz+):   0 found -   1 modified     |    TOTAL:   2
pass   1 (yz-):   2 found -   2 modified     |    TOTAL:   5
pass   1 (yz-):   5 found -   5 modified     |    TOTAL:   7
pass   2 (yz-):   0 found -   2 modified     |    TOTAL:   5
pass   2 (yz-):   0 found -   5 modified     |    TOTAL:   7
pass   1 (xz+):

#### Iterate manual adjustments and doing tesselation & topology fix again
Should look something like this. Keep going (adjust `wm.mgz` using the recon-edit function of Freeview) until you are happy with the result.

![LH_wm_inflated_lateral](pics/LH_wm_inflated_lateral.png) ![LH_wm_inflated_medial](pics/LH_wm_inflated_medial.png)


In [26]:
# look at the result and apply fixes in WM definition
# you will want to get rid of weird 'stalks' and 'bridges'
freeview -v ${fsSurf_mgz}/brain.mgz -v ${fsSurf_mgz}/wm.mgz \
    -f ${fsSurf_temp}/lh.smoothwm ${fsSurf_temp}/lh.inflated \
    ${fsSurf_temp}/rh.smoothwm ${fsSurf_temp}/rh.inflated &

[1] 3245


In [ ]:
# redo the tesselation with the freshly fixed WM volume
mri_fill -CV ${CC[0]} ${CC[1]} ${CC[2]} \
    -PV ${PONS[0]} ${PONS[1]} ${PONS[2]} \
    ${fsSurf_mgz}/wm.mgz ${fsSurf_mgz}/filled.mgz
    
mri_pretess ${fsSurf_mgz}/filled.mgz 255 ${fsSurf_mgz}/brain.mgz ${fsSurf_mgz}/wm_filled-pretess255.mgz &
mri_pretess ${fsSurf_mgz}/filled.mgz 127 ${fsSurf_mgz}/brain.mgz ${fsSurf_mgz}/wm_filled-pretess127.mgz &
wait
mri_tessellate ${fsSurf_mgz}/wm_filled-pretess255.mgz 255 ${fsSurf_temp}/lh.orig &
mri_tessellate ${fsSurf_mgz}/wm_filled-pretess127.mgz 127 ${fsSurf_temp}/rh.orig &
wait

for xh in ${HEMI[@]}; do
    mris_extract_main_component ${fsSurf_temp}/${xh}.orig ${fsSurf_temp}/${xh}.orig
    mris_smooth -nw ${fsSurf_temp}/${xh}.orig ${fsSurf_temp}/${xh}.smoothwm
    mris_inflate ${fsSurf_temp}/${xh}.smoothwm ${fsSurf_temp}/${xh}.inflated
    mris_sphere -q ${fsSurf_temp}/${xh}.inflated ${fsSurf_temp}/${xh}.qsphere
    
    mris_euler_number ${fsSurf_temp}/${xh}.orig
    mris_remove_intersection ${fsSurf_temp}/${xh}.orig ${fsSurf_temp}/${xh}.orig
    mris_smooth -nw ${fsSurf_temp}/${xh}.orig ${fsSurf_temp}/${xh}.smoothwm
    mris_inflate ${fsSurf_temp}/${xh}.smoothwm ${fsSurf_temp}/${xh}.inflated
    mris_curvature -thresh .999 -n -a 5 -w -distances 10 10 ${fsSurf_temp}/${xh}.inflated
done

using voxel position (174, 152, 173) as corpus callosum seed point
using voxel position (174, 226, 139) as pons seed point
reading input volume...done.
searching for cutting planes...INFO: volume does not have linear_transform set nor lta is given by option.-xform.
INFO: use identity matrix as the talairach transform.
INFO: use src volume parameters for the talairach volume.
voxel to talairach voxel transform
 1.00000   0.00000   0.00000   0.00000;
 0.00000   1.00000   0.00000   0.00000;
 0.00000   0.00000   1.00000   0.00000;
 0.00000   0.00000   0.00000   1.00000;
Calculated talairach cc position (-0.25, -7.25, 25.50)
Using the seed to calculate the cutting plane
Looking for area (min, max) = (350, 1400)
area[0] = 1677 (min = 350, max = 1400), aspect = 0.42 (min = 0.10, max = 0.75)
need search nearby
using seed (174, 152, 173), TAL = (-0.2, -7.2, 25.5)
talairach voxel to voxel transform
 1.00000   0.00000   0.00000   0.00000;
 0.00000   1.00000   0.00000   0.00000;
 0.00000   0.00000

In [ ]:
# create the spheres (this takes longer so do it only when you're happy with the inflated)
for xh in ${HEMI[@]}; do
    mris_sphere ${fsSurf_temp}/${xh}.inflated ${fsSurf_temp}/${xh}.sphere
done

#### Create a subject directory in your freesurfer subject directory

In [ ]:
# create a freesurfer SUBJECT with the correct folder structure
# NB! $SUBJECTS_DIR should be defined in your ~/.bashrc as the Freesurfer subjects directory
echo 'Creating a subject directory for '${SUBJ}'in:'
echo ${SUBJECTS_DIR}/${SUBJ}
mksubjdirs ${SUBJECTS_DIR}/${SUBJ}

#### Move files to the Freesurfer subjects directory

In [ ]:
# copy the necessary files for cutting to the subject directory
for xh in ${HEMI[@]}; do
    cp ${fsSurf_temp}/${xh}.inflated ${SUBJECTS_DIR}/${SUBJ}/surf/
    cp ${fsSurf_temp}/${xh}.smoothwm ${SUBJECTS_DIR}/${SUBJ}/surf/
    cp ${fsSurf_temp}/${xh}.orig ${SUBJECTS_DIR}/${SUBJ}/surf/
    cp ${fsSurf_temp}/${xh}.qsphere ${SUBJECTS_DIR}/${SUBJ}/surf/
done

cp ${fsSurf_mgz}/T1.mgz ${SUBJECTS_DIR}/${SUBJ}/mri/T1.mgz
cp ${fsSurf_mgz}/filled.mgz ${SUBJECTS_DIR}/${SUBJ}/mri/filled.mgz
cp ${fsSurf_mgz}/wm.mgz ${SUBJECTS_DIR}/${SUBJ}/mri/wm.mgz
cp ${fsSurf_mgz}/brain.finalsurfs.mgz ${SUBJECTS_DIR}/${SUBJ}/mri/brain.finalsurfs.mgz
cp ${fsSurf_mgz}/brainmask.mgz ${SUBJECTS_DIR}/${SUBJ}/mri/brainmask.mgz

Before we can start making the final surfaces we need a little trick to compensate for the low grey/white matter contrast in monkeys. Now that we have a white matter volume we are happy with (if not go back and fix it) we can use it to enhance the GM/WM contrast artificially and bring the voxel values into the range that Freesurfer expects. The difference looks somewhat like this (Left: normal brain, Right: contrast enhanced brain).

![Brain](pics/brain.png) ![Brain_enh](pics/brain_enhanced.png)

#### Make a contrast-enhanced brain volume to aid in getting the pial surface

In [ ]:
# Make a folder for this procedure and convert the necessary files to nifti so we can use `fslmaths`
mkdir -p $fsSurf/enh
mri_convert ${fsSurf_mgz}/brain.mgz $fsSurf/enh/brain.nii.gz
mri_convert ${fsSurf_mgz}/wm.mgz $fsSurf/enh/wm.nii.gz

# binarize the wm volume
fslmaths $fsSurf/enh/wm.nii.gz -bin $fsSurf/enh/wm_bin.nii.gz

# divide the voxel value in the original brain volume by **some** value and add **another** value to the white matter only 
# which values you need here is up to you but we want to end with white matter voxels having a value ~110 and grey matter ~60-80
# alternative ways are definitely possible as long as your end result is similar
fslmaths $fsSurf/enh/brain.nii.gz -div 2 $fsSurf/enh/brain2.nii.gz
fslmaths $fsSurf/enh/wm_bin.nii.gz -mul 20 $fsSurf/enh/wm_add.nii.gz
fslmaths $fsSurf/enh/brain2.nii.gz -add $fsSurf/enh/wm_add.nii.gz $fsSurf/enh/brain_enh.nii.gz

# convert the 'enhanced' brain back to mgz
mri_convert $fsSurf/enh/brain_enh.nii.gz ${SUBJECTS_DIR}/${SUBJ}/mri/brain_enh.mgz

#### Create the surfaces

In [ ]:
# create surfaces
# this will take a while. don't wait for it. go get a coffee, call your mother, or read a paper.
for xh in ${HEMI[@]}; do
    mris_make_surfaces -noaseg -noaparc -T1 brain_enh -orig_wm orig ${SUBJ} ${xh}
    mris_sphere ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.inflated ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.sphere
done

You should now have nicely looking inflated white matter surfaces and a projection of it onto a sphere surface. They should look somewhat like the following images.
`freeview -f ${fsSurf_temp}/lh.smoothwm ${fsSurf_temp}/lh.inflated ${fsSurf_temp}/lh.sphere &`

![surfaces_leged](pics/final_surf_legend.png)

In [ ]:
# freeview -f ${fsSurf_temp}/lh.smoothwm ${fsSurf_temp}/lh.inflated ${fsSurf_temp}/lh.sphere &
freeview -f ${SUBJECTS_DIR}/${SUBJ}/surf/lh.smoothwm ${SUBJECTS_DIR}/${SUBJ}/surf/lh.inflated ${SUBJECTS_DIR}/${SUBJ}/surf/lh.sphere &

In [ ]:
# check and edit the pial surface
mkdir -p ${SUBJECTS_DIR}/${SUBJ}/pial_edits # create a folder for pial edits
cp -R ${SUBJECTS_DIR}/${SUBJ}/mri ${SUBJECTS_DIR}/${SUBJ}/pial_edits/ 
cp -R ${SUBJECTS_DIR}/${SUBJ}/surf ${SUBJECTS_DIR}/${SUBJ}/pial_edits/

In [ ]:
freeview -v ${SUBJECTS_DIR}/${SUBJ}/pial_edits/mri/T1.mgz ${SUBJECTS_DIR}/${SUBJ}/pial_edits/mri/brain_enh.mgz \
    -f ${SUBJECTS_DIR}/${SUBJ}/pial_edits/surf/lh.white:edgecolor=yellow ${SUBJECTS_DIR}/${SUBJ}/pial_edits/surf/lh.pial:edgecolor=red \
    ${SUBJECTS_DIR}/${SUBJ}/pial_edits/surf/rh.white:edgecolor=yellow ${SUBJECTS_DIR}/${SUBJ}/pial_edits/surf/rh.pial:edgecolor=red &

In [ ]:
# NB! If you adjustments to the brainmask (which probably isn't necessary if NMT segmentation worked well) 
# don't forget to copy back the adjusted brain_enh for re-generation of the pial surface 
cp ${SUBJECTS_DIR}/${SUBJ}/pial_edits/mri/brain_enh.mgz ${SUBJECTS_DIR}/${SUBJ}/mri/brain_enh.mgz

In [ ]:
# regenerate the pial surface
for xh in ${HEMI[@]}; do
    mris_make_surfaces  -noaseg -noaparc -orig_white white -orig_pial white -nowhite -mgz -T1 brain_enh ${SUBJ} ${xh}
done

In [ ]:
# create midcortical surface by growing the white matter halfway towards the pial surface
for xh in ${HEMI[@]}; do
    mris_expand -thickness ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.white 0.5 ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.graymid
done

In [ ]:
# Fix topology, smooth & inflate the final surfaces
for xh in ${HEMI[@]}; do    
    mris_euler_number ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.white
    mris_remove_intersection ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.white ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.white
    mris_smooth -n 3 -nw ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.white ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.smoothwm
    
    mris_euler_number ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.pial
    mris_remove_intersection ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.pial ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.pial
    mris_smooth -n 3 -nw ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.pial ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.smoothpial
    
    mris_euler_number ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.graymid
    mris_remove_intersection ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.graymid ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.graymid
    mris_smooth -n 3 -nw ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.graymid ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.smoothgraymid
    
    mris_inflate ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.smoothwm ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.inflated
    mris_curvature -thresh .999 -n -a 5 -w -distances 10 10 ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.inflated
    mris_sphere ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.inflated ${SUBJECTS_DIR}/${SUBJ}/surf/${xh}.sphere
done

#### Cut surfaces and create flatmaps

The cells below will load the inflated hemispheres in tksurfer where you can make the cuts to create the flatmaps.  

For the full hemisphere make cuts on the medial wall. One cut enclosing the midline and five additional relaxation cuts.
Mark a point on the part of the surface you want to keep and save patch as `?h.full.patch.3d` (replace '?' with 'l' and 'r' respectively)

![Cut_full](pics/Cut_full.png)

For an occipital patch make one cut on the medial wall along the calcarine sulcus. Use 3 points to select a coronal cutting plane, and a fourth point to select which part of the surface you want to keep and save as `?h.occip.patch.3d`

![Cut_occip1](pics/Cut_occip1.png) ![Cut_occip1](pics/Cut_occip2.png)

In [ ]:
# left
#tksurfer ${SUBJ} lh inflated -curv
tksurfer ${SUBJ} lh inflated -gray 

In [ ]:
# right
#tksurfer ${SUBJ} rh inflated -curv
tksurfer ${SUBJ} rh inflated -gray 

In [ ]:
# Time for another coffee, you are not going to want to wait for this to finish while staring at its progress
cd ${SUBJECTS_DIR}/${SUBJ}/surf/
for xh in ${HEMI[@]}; do
    mris_flatten -w 0 -distances 20 7 ${xh}.full.patch.3d  ${xh}.full.patch.flat
    mris_flatten -w 0 -distances 20 7 ${xh}.occip.patch.3d  ${xh}.occip.patch.flat
done

If everything went well, you now have flatmaps of the white matter surface. 
They should look somewhat like this:  

**Full hemisphere (rh)**  
![RH_flatmap](pics/RH_Flat_Sulci_gray_nolegend.png) ![RH_flatmap_legend](pics/RH_Flat_Sulci_gray_legend.png)

**Occipital (rh)** (functional areas are guesstimates based on anatomy)  
![RH_occip](pics/RH_Flat_occip_gray_nolegend.png) ![RH_occip_gray_legend](pics/RH_Flat_occip_gray_legend.png)

It is of course also possible to make **any** other funtionally relevant cutting pattern to extract a flatmap of a piece of cortex. The method is the same.

In [ ]:
# check your result
#tksurfer ${SUBJ} rh inflated -patch rh.full.patch.flat -curv # for a red/green curvature map
tksurfer ${SUBJ} lh inflated -patch lh.full.patch.flat -gray # for a gray curvature map

#### Convert Freesurfer surfaces to Gifti
Other software may like this better.

In [ ]:
mkdir -p gii
for xh in ${HEMI[@]}; do
    mris_convert ${xh}.white ./gii/${xh}.white.gii
    mris_convert ${xh}.graymid ./gii/${xh}.graymid.gii
    mris_convert ${xh}.pial ./gii/${xh}.pial.gii
    mris_convert ${xh}.inflated ./gii/${xh}.inflated.gii

    mris_convert ${xh}.smoothwm ./gii/${xh}.smoothwm.gii
    mris_convert ${xh}.smoothgraymid ./gii/${xh}.smoothgraymid.gii
    mris_convert ${xh}.smoothpial ./gii/${xh}.smoothpial.gii
    
    mris_convert -p -c ${xh}.curv ${xh}.full.patch.flat ./gii/${xh}.full.patch.flat.gii
    mris_convert -p -c ${xh}.curv ${xh}.occip.patch.flat ./gii/${xh}.occip.patch.flat.gii
done

#### Copy Freesurfer results back to NMT template folder

In [ ]:
cp -r ${SUBJECTS_DIR}/${SUBJ}/surf ${fsSurf}/surf

In [ ]:
# return to startpath
cd ${startpath}

----
----
## **Step 4 (OPTIONAL): Create additional surfaces using Connectome Workbench.**
This should not be necessary at all. Try the Freesurfer approach and a conversion to gifti with `mris_convert` first (as explained above)  

----
----

### Requirements
  - Connectome Workbench (https://www.humanconnectome.org/software/connectome-workbench)  

In [ ]:
# THIS SHOULDN'T BE NECESSARY SINCE WE'RE GETTINGS THESE OUTPUTS ALSO FROM THE FREESURFER PIPELINE !! #

# set path to where you want the results
wbSurf_path=${NMT_path}/single_subject_scans/${SUBJ}/wbSurf

echo 'Extracting surfaces from segmentation'
cd ${wbSurf_path}

# create surfaces
IsoSurface -input ${NMT_ss_path}/NMT_${SUBJ}_process/${SUBJ}_segmentation.nii.gz \
    -isorois -o_gii surf

# rename to something recognizable
mv surf.k2.gii gm.surf.gii # gm surface
mv surf.k3.gii wm.surf.gii # wm surface

# Smooth the surfaces a bit
echo 'Smoothing the surfaces a bit' 
wb_command -surface-smoothing wm.surf.gii 0.5 1 wm_sm.surf.gii
wb_command -surface-smoothing gm.surf.gii 0.5 1 gm_sm.surf.gii

# inflate the surfaces
echo 'Inflating the surfaces'
wb_command -surface-generate-inflated \
    wm_sm.surf.gii infl_wm_sm.surf.gii vinfl_wm_sm.surf.gii
wb_command -surface-generate-inflated \
    gm_sm.surf.gii infl_gm_sm.surf.gii vinfl_gm_sm.surf.gii

# view the result in your favorite viewer
echo 'Inspect the surfaces in a viewer'
freeview -f wm_sm.surf.gii infl_wm_sm.surf.gii vinfl_wm_sm.surf.gii \
    gm_sm.surf.gii infl_gm_sm.surf.gii vinfl_gm_sm.surf.gii